# Seasonal water sub-model - Notebook 4

* **Products used:** 
[cgls_landcover](https://explorer.digitalearth.africa/products/cgls_landcover)

## Background

This notebook calculates and stores the first sub-model in the elephant Habitat Suitability Model; the other vegetation sub-model. It has the data layers and weightings listed in the table below.

The notebook executes the following steps:

   1. Define an area of interest and load packages.
   2. Load landcover CGLS data and select the cropland and urban cover layers.
   3. Within the area of interest, calculate proximity to croplands and urban areas quantified as euclidean distance in metres.
   4. Standardise the euclidean distance measure so it ranges between 0 and 1.
   5. Use a weighted linear combination approach to estimate habitat suitability.
   6. Visualise the habitat suitability model.

### Load packages
Import Python packages that are used for the analysis.

In [1]:
%matplotlib inline

import datacube
import os
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import rioxarray as rxr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from odc.algo import xr_reproject
from deafrica_tools.plotting import display_map, plot_lulc
from deafrica_tools.spatial import xr_vectorize, xr_rasterize

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
try:
      import xrspatial
except:
      !pip install xrspatial
    
from xrspatial import proximity

/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


### Connect to the datacube

Connect to the datacube so we can access DE Africa data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [3]:
dc = datacube.Datacube(app='Elephant-HSM-sw')

## Load CGLS landcover

Spatial parameters must be the same as other notebooks.

In [4]:
%store -r lons
%store -r lats

query = {'x': lons,
         'y': lats,
         'resolution': (-200, 200),
         'time': '2019',
         'output_crs': 'epsg:6933',
         'measurements': 'classification'}
ds_cgls = dc.load(product='cgls_landcover', **query).squeeze()

ds_cgls

<xarray.Dataset>
Dimensions:         (y: 3618, x: 2895)
Coordinates:
    time            datetime64[ns] 2019-07-02T11:59:59.500000
  * y               (y) float64 -2.047e+06 -2.048e+06 ... -2.770e+06 -2.771e+06
  * x               (x) float64 1.416e+06 1.417e+06 ... 1.995e+06 1.995e+06
    spatial_ref     int32 6933
Data variables:
    classification  (y, x) uint8 126 126 126 20 126 20 126 ... 20 20 20 20 20 20
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref

# Visualise location of seasonal water

In [5]:
cgls_sw = ds_cgls.where((ds_cgls.seasonalwater_cover_fraction>0) & (ds_cgls.seasonalwater_cover_fraction<=100))
cgls_sw.seasonalwater_cover_fraction.plot()

AttributeError: 'Dataset' object has no attribute 'seasonalwater_cover_fraction'

In [ ]:
proximity_sw = proximity(cgls_sw.seasonalwater_cover_fraction, distance_metric = 'EUCLIDEAN')

In [ ]:
sw_proximity_std = 1 - (proximity_sw/np.max(proximity_sw))
sw_proximity_std = sw_proximity_std.fillna(0)
sw_proximity_std.plot()

## Save to outputs

In [ ]:
sw_model = sw_proximity_std.drop('time', dim=None)
sw_model.to_netcdf('outputs/sw_sm.nc')

---

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:**

In [ ]:
print(datacube.__version__)

**Last Tested:**

In [ ]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')